In [19]:
import glob
import os

import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv

In [14]:
urls = {"0": "https://www.gutenberg.org/files/25326/25326-h/25326-h.htm",
        "1": "https://www.gutenberg.org/files/25759/25759-h/25759-h.htm",
        "2": "https://www.gutenberg.org/cache/epub/26860/pg26860-images.html",
        "3": "https://www.gutenberg.org/cache/epub/28420/pg28420-images.html",
        "4": "https://www.gutenberg.org/cache/epub/28421/pg28421-images.html",
        "5": "https://www.gutenberg.org/cache/epub/28422/pg28422-images.html",
        "6": "https://www.gutenberg.org/cache/epub/31845/pg31845-images.html",
        "7": "https://www.gutenberg.org/cache/epub/31938/pg31938-images.html",
        "8": "https://www.gutenberg.org/cache/epub/32362/pg32362-images.html",
        "9": "https://www.gutenberg.org/cache/epub/33203/pg33203-images.html"}

In [15]:
# Function to scrape the HTML page and extract page ranges
def scrape_and_extract_page_ranges(url, volume):
    volume_label = "CONTENTS_OF_VOLUME_" + volume
    
    print(volume_label)
    page_ranges = {}
    # Send a GET request to the URL
    response = requests.get(url)

    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all <p> tags
    table_elements = soup.find(id=volume_label).findNext('div').find('table').find_all('tr')

    # Iterate over rows
    for row in table_elements:
        # Find all cells in the row
        cells = row.find_all(['td', 'th'])
        # Extract artist name and page number
        if len(cells) >= 2:
            if cells[1].get_text().strip().isdigit():
                artist_name = cells[0].get_text().strip()
                page_number = cells[1].get_text().strip()
                # Find next row to get the end page of the range
                next_row = row.find_next_sibling('tr')
                if next_row:
                    next_cells = next_row.find_all(['td', 'th'])
                    if len(next_cells) >= 2 and next_cells[1].get_text().strip().isdigit():
                        end_page_number = int(next_cells[1].get_text().strip())-1
                        page_ranges[artist_name] = f"{page_number}-{end_page_number}"
            else:
                artist_name = cells[0].get_text().strip()
                print(artist_name)
                page_number = cells[1].get_text().strip()
                page_ranges[artist_name] = f"{page_number}"

    return page_ranges


In [16]:
for volume, url in urls.items():
    pr = scrape_and_extract_page_ranges(url=url, volume=volume)
    directory = f'{volume}'
    if not os.path.exists(directory):
        os.makedirs(directory)
    # Write results to CSV
    with open(f'{directory}/biography_page_ranges_' + volume + '.csv', 'w', newline='') as csvfile:
        fieldnames = ['Name', 'Page Range']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
        writer.writeheader()
        for name, page_range in pr.items():
            writer.writerow({'Name': name, 'Page Range': page_range})

CONTENTS_OF_VOLUME_0


KeyboardInterrupt: 

In [21]:
for file in glob.glob('../../data/biographies/ranges/*.csv'):
    df = pd.read_csv(file)

ParserError: Error tokenizing data. C error: Expected 2 fields in line 12, saw 3
